# Building trees from line list of HIV infections simulated with a branching process

This notebook shows how to generate trees based on the line lists output by the HIV branching process simulator. The trees are generated using functions in IDM's `phylomodels` library. They are formatted as `ETE3` objects.

We first load the set of packages that allow us to run the simulator:

In [1]:
# Standard packages
import os
import numpy as np
import pandas as pd

%matplotlib inline

# R-related packages
import rpy2
import rpy2.robjects as robjects
r = robjects.r

from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# We may need to install some packages
try:
    from rpy2.robjects.packages import importr
    dplyr = importr('dplyr')
except RRuntimeError:
    from rpy2.robjects.packages import importr, data
    utils = importr('utils')
    base = importr('base')
    utils.chooseCRANmirror()
    utils.install_packages('dplyr')

# Set up working directory
cwd = os.getcwd()
os.chdir('../')

In [2]:
r.source('hiv_branching_process.R')

value,[RTYPES.NILSXP]
visible,[RTYPES.LGLSXP]


Now we can configure and run the simulator. Once we do that, we can convert the R objects returned by `simulate_transmission` into pandas dataframes. In particular, we are interested in the `population_summary` dataframe, which is the linelist that will be used for building the transmission trees.

In [3]:
out = r.simulate_transmission( sim_time=365*10 )
population_summary_r  = out.rx2('population_summary' )
transmission_record_r = out.rx2('transmission_record')

with localconverter( robjects.default_converter + pandas2ri.converter ):
    population_summary  = robjects.conversion.rpy2py( population_summary_r  )
    transmission_record = robjects.conversion.rpy2py( transmission_record_r )

  |==================================================| 100%


In [4]:
population_summary.head()

,recipient,removal_rate,partners,acts_per_day,transmission_risk_per_act,transmission_risk_per_day,source,infectionTime,sampleTime,cumulative_partners,cumulative_transmissions
1,1.0,0.001,0,0.152823,0.001549,0.000000,0.0,0.0,365.0,0.0,0.0
2,2.0,0.001,1,0.072246,0.001522,0.000110,0.0,0.0,365.0,0.0,0.0
3,3.0,0.001,0,0.522873,0.002059,0.000000,0.0,0.0,365.0,0.0,0.0
4,4.0,0.001,2,0.854449,0.001926,0.003289,0.0,0.0,684.0,0.0,2.0
5,5.0,0.001,0,0.577191,0.001668,0.000000,0.0,0.0,365.0,0.0,0.0


We can now build the trees and work with them as we would work with any `ETE3` tree object: we can draw the tree, save it as a newick file, sample it, etc.

In [5]:
from phylomodels.trees import generate_treeFromFile
from phylomodels.trees.transform_transToPhyloTree import transform_transToPhyloTree
from phylomodels.trees.transform_joinTrees import transform_joinTrees

In [6]:
# Some formatting to make things easier
population_summary['recipient'] = population_summary['recipient'].astype(int).astype(str)
population_summary['source'] = population_summary['source'].astype(int).astype(str)

In [7]:
# Let's get rid of seed infections that didn't produce other infections
all_seeds = population_summary[ population_summary['source']=='0' ].index
all_infected_by_seed = population_summary[ population_summary['source'].isin( all_seeds ) ]
all_successful_seeds = all_infected_by_seed['source'].unique()
all_unsuccessful_seeds = list( set(all_seeds) - set(all_successful_seeds) )
population_summary_lean = population_summary.drop( index=all_unsuccessful_seeds )

In [8]:
# We first build a raw tree from the population summary line list, but this tree is unrooted and not phylo)
trees = generate_treeFromFile.read_treeFromLineList( population_summary_lean,
                                                     ID = 'recipient',
                                                     infectorID = 'source',
                                                     infectTime = 'infectionTime',
                                                     sampleTime = 'sampleTime',
                                                     features = ['partners', 'acts_per_day', 'transmission_risk_per_act', 'removal_rate']
                                                   )
raw_tree = trees[0]
raw_tree.describe()

Number of leaf nodes:	3545
Total number of nodes:	4181
Rooted:	No
Most distant node:	4260
Max. distance:	3646.000000


In [9]:
# We now build phylo trees (i.e., bifurcating trees with sampled nodes as trees)
seeds = raw_tree.get_children()
seeds_phylo = []
for this_seed in seeds:
    seed_tree = transform_transToPhyloTree( this_seed )
    seeds_phylo.append( seed_tree )

In [10]:
# And coalesce into a single tree
tree = transform_joinTrees( seeds_phylo )
tree.describe()

Number of leaf nodes:	4180
Total number of nodes:	8359
Rooted:	Yes
Most distant node:	4012
Max. distance:	16505.702299


In [11]:
tree.render('%%inline')